[](http://)

In [ ]:
# bbox-utility, check https://github.com/awsaf49/bbox for source code
!pip install -q /kaggle/input/loguru-lib-ds/loguru-0.5.3-py3-none-any.whl
!pip install -q /kaggle/input/bbox-lib-ds

In [ ]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index
%cd ..

#  Import Libraries

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image

## Please Upvote if you find this Helpful

#  Meta Data
* `train_images/` - Folder containing training set photos of the form `video_{video_id}/{video_frame}.jpg`.

* `[train/test].csv` - Metadata for the images. As with other test files, most of the test metadata data is only available to your notebook upon submission. Just the first few rows available for download.

* `video_id` - ID number of the video the image was part of. The video ids are not meaningfully ordered.
* `video_frame` - The frame number of the image within the video. Expect to see occasional gaps in the frame number from when the diver surfaced.
* `sequence` - ID of a gap-free subset of a given video. The sequence ids are not meaningfully ordered.
* `sequence_frame` - The frame number within a given sequence.
* `image_id` - ID code for the image, in the format `{video_id}-{video_frame}`
* `annotations` - The bounding boxes of any starfish detections in a string format that can be evaluated directly with Python. Does not use the same format as the predictions you will submit. Not available in test.csv. A bounding box is described by the pixel coordinate `(x_min, y_min)` of its lower left corner within the image together with its `width` and `height` in pixels --> (COCO format).

###### ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# CKPT_DIR  = '/kaggle/input/greatbarrierreef-yolov5-train-ds'
CKPT_PATH = '/kaggle/input/leonv5inferv5s60/best.pt' # by @steamedsheep
IMG_SIZE  = 6400
CONF      = 0.30
IOU       = 0.50
AUGMENT   = True

In [ ]:
# Train Data
df = pd.read_csv(f'/kaggle/input/tensorflow-great-barrier-reef/train.csv')
df['image_path'] = f'/kaggle/input/tensorflow-great-barrier-reef/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))


## Number of BBoxes

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

#  Helper

In [ ]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo, voc2coco
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

# Inference

## Helper

In [ ]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

## Tracker

In [ ]:
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)


def tracking_function(tracker, frame_id, bboxes, scores):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id)) #tracker.updateは検出したオブジェクトを返す。#オプションのdetectionshは処理中の現在のフレームで検出された検出物を表すDetectionのリスト。
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    return predictions

## Run Inference on **Train**

In [ ]:
CKPT_PATH = '../input/yolov5s6/f2_sub2.pt'
CONF = 0.30 #0.30
IOU = 0.50
IMG_SIZE = 6400
AUGMENT = False
Alb=False

In [ ]:
pwd

In [ ]:
from PIL import Image
import ast
import albumentations as albu

In [ ]:
Trans = albu.Compose([
    #albu.RandomCrop(width=128, height=72),
    #albu.HorizontalFlip(p=0.5),
    #albu.RandomBrightnessContrast(p=1.0),
    albu.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.5),
    albu.RGBShift (r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, always_apply=False, p=0.5),
    #albu.Normalize (mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ##albu,ChannelDropout (channel_drop_range=(1, 1), fill_value=0, always_apply=False, p=0.5),
    #albu.ChannelShuffle(p=1.0),
    #albu.RandomRotate90(p=0.5),
    #albu.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8),p=1.0), #(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)
    #albu.FDA(FDA_reference['image_path'].values,p=0.5),
    #albu.Affine(p=1.0), #(scale=None, translate_percent=None, translate_px=None, rotate=None, shear=None, interpolation=1, mask_interpolation=0, cval=0, cval_mask=0, mode=0, fit_output=False, always_apply=False, p=0.5)
    #albu.CenterCrop(128,72,p=0.5), #(height, width, always_apply=False, p=1.0)
    #albu.RandomCropNearBBox(), #(max_part_shift=(0.3, 0.3), cropping_box_key='cropping_bbox', always_apply=False, p=1.0)
    #albu.RandomRotate90(p=0.5),
    #albu.RandomResizedCrop(p=0.5), #(height, width, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=1.0)
    #albu.GaussianBlur(), #(blur_limit=(3, 7), sigma_limit=0, always_apply=False, p=0.5)
    #albu.GaussNoise(), #(var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5
    #albu.Blur(), #(blur_limit=7, always_apply=False, p=0.5)
    #albu.
    
])

In [ ]:
def CLAHE(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)
    return equalized
def Gamma_enhancement(image):
    gamma = 1/0.6
    R = 255.0
    return (R * np.power(image.astype(np.uint32)/R, gamma)).astype(np.uint8)

## repos enhancment

In [ ]:
def RecoverCLAHE(sceneRadiance):
    clahe = cv2.createCLAHE(clipLimit=7, tileGridSize=(14, 14))
    for i in range(3):

        
        sceneRadiance[:, :, i] = clahe.apply((sceneRadiance[:, :, i]))


    return sceneRadiance

#GC
def RecoverGC(sceneRadiance):
    sceneRadiance = sceneRadiance/255.0
    
    for i in range(3):
        sceneRadiance[:, :, i] =  np.power(sceneRadiance[:, :, i] / float(np.max(sceneRadiance[:, :, i])), 3.2)
    sceneRadiance = np.clip(sceneRadiance*255, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)
    return sceneRadiance

In [ ]:
import numpy as np

def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L

def stretching(img):
    height = len(img)
    width = len(img[0])
    for k in range(0, 3):
        Max_channel  = np.max(img[:,:,k])
        Min_channel  = np.min(img[:,:,k])
        for i in range(height):
            for j in range(width):
                img[i,j,k] = (img[i,j,k] - Min_channel) * (255 - 0) / (Max_channel - Min_channel)+ 0
    return img

from skimage.color import rgb2hsv,hsv2rgb
import numpy as np



def  HSVStretching(sceneRadiance):
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)

    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    

    return img_rgb

def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance

In [ ]:
#ICM
def RecoverICM(img1):
    img = stretching(img1)
    sceneRadiance = sceneRadianceRGB(img)
    sceneRadiance = HSVStretching(sceneRadiance)
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    
    return sceneRadiance

In [ ]:
image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()

In [ ]:
Debug = True

In [ ]:
Alb=False

## Norfair Tracker　の説明

hit_inertia_min（オプション）：各追跡オブジェクトは、検知にマッチする頻度を追跡する内部ヒットイナーシャカウンターを保持しており、マッチするたびにこのカウンターが上がり、マッチしないたびに下がります。あるフレーム数の間マッチングがなく、この引数で設定された値以下になった場合、オブジェクトは破棄されます。デフォルトは10です。


hit_inertia_max (オプション)。各追跡オブジェクトは、検知にマッチする頻度を追跡する内部ヒットイナーシャカウンターを保持しており、マッチするたびにこのカウンターが上がり、マッチしないたびに下がります。この引数は、この慣性力をどれだけ大きくできるかを定義し、したがって、オブジェクトがどの検知にもマッチしない状態でどれだけ長く生きることができるかを定義します。デフォルトは25です。

initialization_delay (オプション)。各追跡オブジェクトは、その内部ヒット慣性カウンタがhit_inertia_minを超えるまで、トラッカーがユーザーに返す潜在的なオブジェクトとして考慮されるのを待ちます。initialization_delayは、オブジェクトのヒットイナーシャカウンターがhit_inertia_minをどれくらい超えれば、初期化されたとみなされ、実際のオブジェクトとしてユーザーに返されるかを決定する引数です。デフォルトは (hit_inertia_max - hit_inertia_min) / 2 です。


detection_threshold (オプション)。トラッカーに送り込まれる検出ポイントのスコアが、トラッカーに無視されるために、それ以下に落ち込まなければならない閾値を設定します。デフォルトは 0 です。

point_transience (オプション)。各追跡オブジェクトは、追跡しているポイントがどれくらいの頻度でマッチングしているかを追跡しています。マッチングされているポイントはライブ、そうでないポイントはライブでないと言われています。これは、追跡オブジェクトのどの点が draw_tracked_objects によって描画され、どの点が描画されないかというようなことを決定します。この引数は、マッチングされないポイントがどれだけ短命かを決定します。デフォルトは 4 です。


filter_setup (オプション)。このパラメータは、TrackedObjectインスタンスが使用するKalman Filterのパラメータを変更するために使用します。デフォルトはFilterSetup()です。

past_detections_length: 各追跡オブジェクトについて、保存する過去の検出回数を指定します。Norfairは、オブジェクトのライフタイムを通じて、これらの過去の検出を均一に分配しようとするので、オブジェクトをより代表的にすることができます。各検出に埋め込みを関連付け、距離関数にアクセスできるため、モデルに計量学習を追加したい場合に非常に便利です。デフォルトは4です。

In [ ]:
np.random.seed(32)
if Debug:
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30, #30
        hit_inertia_min=3, #3
        hit_inertia_max=6, #6
        initialization_delay=1, #1 追跡対象の各オブジェクトは、内部ヒットインターティアカウンタが、トラッカーによってユーザーに返される可能性のあるオブジェクトとして見なされるまで待機します。この引数は、オブジェクトのヒット慣性カウンタが初期化済みとみなし、実際のオブジェクトとしてユーザーに返されるのにどれくらいの大きな値を超えなければならないかによって決まります。デフォルトは にします。
        detection_threshold =0, #0 トラッカーに供給される検出のポイントのスコアがトラッカーによって無視されるように下に浸す必要があるしきい値を設定します。デフォルトは にします。0
        point_transience=4,#4 各追跡対象オブジェクトは、そのトラッキングがどの程度の頻度で一致しているかを追跡します。試合を受けているポイントはライブと言われ、そうでないポイントはライブではないと言われます。これは、トラックされたオブジェクトのどの点がdraw_tracked_objectsによって描画されるか、どの点が描画されないかを決定します。この引数は、一致しない短い一生ポイントがどの程度短いかを決定します。デフォルトは にします。4
        filter_setup=FilterSetup(R=100, #100 センサ測定ノイズマトリックスの乗数,   自分で変数をいじったりできるやつ。optional): このパラメータを使用して、TrackedObject インスタンスで使用されるカルマン フィルタのパラメータを変更できます。デフォルトはフィルタ設定()です。
                                 Q=0.1,  #0.1 プロセスの不確実性の乗数
                                 P=10)   #10 初期共分散行列推定の乗数(位置(速度ではない)変数に対応するエントリのみ
        #past_detections_length=4 #: 追跡対象のオブジェクトごとに保存する過去の検出の数。Norfairは、オブジェクトの有効期間を通じてこれらの過去の検出を一様に配布しようとします。各検出に埋め込みを関連付け、距離関数でアクセスできるため、モデルにメトリック学習を追加する場合に非常に便利です。デフォルトは にします。4
        
        
    )

    model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
    frame_id = 0

    for idx, path in enumerate(image_paths):
        img = cv2.imread(path)[...,::-1]
        if Alb:
            #img = RecoverCLAHE(img) #意外といい感じ。学習にこれいいかも
            #img = RecoverGC(img) #ぜんぜんだめ #ごみかす
            #img = RecoverICM(img) #見た感じはいいが誤検出や検出しないのがある。学習に使えば変わるのかもしれないが時間ないからためせない。
            img = Trans(image=img)['image']
            #img = CLAHE(img) #白くなって見落としが増えたからダメ。
            #img = Gamma_enhancement(image=img) #RecoverICMと同じ感じ
        bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
        predict_box = tracking_function(tracker, frame_id, bboxes, confis)

        if len(predict_box)>0:
            box = [list(map(int,box.split(' ')[1:])) for box in predict_box]
        else:
            box = []
        display(show_img(img, box, bbox_format='coco'))
        #display(show_img(img, bboxes, bbox_format='coco'))
        if idx>20:
            break
        frame_id += 1


### past_detections_length

## Init `Env`

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

## Run Inference on **Test**

In [ ]:
%cd ../working

In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30, #距離の閾値である #30
    hit_inertia_min=3, #あるフレーム数の間マッチングがなく、この引数で設定された値以下になった場合、オブジェクトは破棄されます
    hit_inertia_max=6, #この引数は、この慣性力をどれだけ大きくできるかを定義し、したがって、オブジェクトがどの検知にもマッチしない状態でどれだけ長く生きられるかを定義します。
    initialization_delay=1,
)

frame_id = 0
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    if Alb:
        #img = Trans(image=img)['image']
        img = RecoverICM(img)
        
    bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    predictions = tracking_function(tracker, frame_id, bboxes, confs)
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)
    
    if frame_id < 3:
        if len(predictions)>0:
            box = [list(map(int,box.split(' ')[1:])) for box in predictions]
        else:
            box = []
        display(show_img(img, box, bbox_format='coco'))
    print('Prediction:', pred_df)
    frame_id += 1

#  Check Submission

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

In [ ]:
print("all done")